In [ ]:
from CQEDModel import *
from Pulses import *
import qutip as qp
dur=1.0
fm=100.0
na=0.05
tlist=np.linspace(-3*dur,6*dur,400)
tlist0=np.linspace(-5*dur,6*dur,20000)

'''Initialize all pulse objects'''
#Pump pulse
pulse1=Pulse(amplitude=18.0, frequency=0, phase=0, pulse_type="Gaussian", duration=dur, center=dur, noise=generate_pulse_noise\
             ,noiseamp=na,freqmult=fm,sq=False)
#pulse1.precomputing_noise(tlist0)

'''Stokes pulse'''
pulse2=Pulse(amplitude=18.0, frequency=0, phase=0, pulse_type="Gaussian", duration=dur, center=-dur, noise=generate_pulse_noise\
             ,noiseamp=na,freqmult=fm,sq=False)
#pulse2.precomputing_noise(tlist0)

Delta0=400.0
Pa=80.0
#Delta0=5000
#Pa=173.205

'''Classical pulse contribution to AC Stark shift''' 
APulse=Pulse(amplitude=Pa, frequency=0, phase=0, pulse_type="Gaussian", duration=dur, center=dur, noise=generate_pulse_noise
             ,noiseamp=na,freqmult=fm,sq=True)
#APulse.precomputing_noise(tlist0)


'''Xontrol pulse sequence to negate two-photon detuning shift due to AC Stark shift'''
cdur=dur/np.sqrt(2.0)
CPulse=Pulse(amplitude=-1/2*Pa**2/Delta0, frequency=0, phase=0, pulse_type="Gaussian",\
             duration=cdur, center=dur, noise=generate_pulse_noise,noiseamp=na,freqmult=fm)
#CPulse.precomputing_noise(tlist0)
pulse1.plot_pulse(tlist)

options1 = {"progress_bar": True, "map": "parallel"}
#options1 = {'progress_bar': True,'map': 'mpi','num_cpus': 40}

'''Initialize N-qutrit+1 cavity mode system'''
Dc1=DickeRaman(Nbits=3, Ncav=2, Det=48.0, tdet=0.0, spem=True,cavdecay=False)
Dc1.construct_ham(pulse1, pulse2); #STIRAP interaction hamiltonian with pulse objects
Dc1.inject_photon(npx=1.0,coherent = False); #Inject npx photons into cavity

'''Dynamic decoupling sequence to cancel drift'''
Dc1.introduce_globalpiswap(3.0*dur,0.01)
Dc1.introduce_globalpiswap(6.0*dur,0.01)

Dc1.apply_dissipation(1.0,1.0,0.05); #Dissipation terms
Dc1.apply_ACdispersive(Pa,APulse,Delta0); #AC Stark shift terms
Dc1.apply_controlfield(CPulse); #Control field to suppress AC Stark shift
display(Dc1.check_ACeffect()) #Check AC Stark to lower 2 levels is gone

Dc1.introduce_drift(driftA=0.05); #Drift Hamiltonian (constant randomly assigned frequency shifts)
Dc1.introduce_freqnoise(system="atomic",noise_type="Deterministic",sigma=[0.0,0.0,0.0],noise_freq=100.0);
#Additional continuous frequency drift 

states=[]
st1=Dc1.simulate_mcwf(tlist,num_cpus=60, ntraj=1000,options=options1).states
#st1=Dc1.simulate(tlist).states
dicke2=generate_dicke_state(3, 1, 2)
max([fidelity(x,dicke2) for x in st1])

In [ ]:
import numpy as np
import concurrent.futures

dicke2 = generate_dicke_state(3, 1, 2)

def run_simulation(ka, dr):
    """Function to run a single simulation for a given `ka` and `dr`."""
    fl = np.array([])
    for a in range(5):
        Dc1 = DickeRaman(Nbits=3, Ncav=2, Det=48.0, tdet=0.0, spem=True, cavdecay=False)
        Dc1.construct_ham(pulse1, pulse2)
        Dc1.apply_dissipation(1.0, 1.0, ka)
        Dc1.inject_photon(npx=1.0, coherent=False)
        Dc1.apply_ACdispersive(Pa, APulse, Delta0)
        Dc1.apply_controlfield(CPulse)
        Dc1.introduce_globalpiswap(3.0 * dur, 0.01)
        Dc1.introduce_globalpiswap(6.0 * dur, 0.01)
        Dc1.introduce_drift(driftA=dr)
        Dc1.introduce_freqnoise(system="atomic", noise_type="Deterministic", sigma=[0.0, 0.0, 0.0], noise_freq=100.0)
        
        # Simulate and calculate fidelity
        #st1 = Dc1.simulate(tlist).states
        st1 = Dc1.simulate_mcwf(tlist,num_cpus=60, ntraj=1000,options=options1).states
        fl = np.append(fl, max([fidelity(x, dicke2) for x in st1]))
        st1 = []
    
    return np.min(fl)

def parallelize_simulations(kappalist, drlist):
    """Function to parallelize the nested loops."""
    # Prepare a list to hold all tasks (each combination of `ka` and `dr`)
    tasks = [(ka, dr) for ka in kappalist for dr in drlist]
    
    # Initialize an empty list to store the results
    fidelitylist = np.array([])
    
    # Use ProcessPoolExecutor to parallelize the tasks
    with concurrent.futures.ProcessPoolExecutor(max_workers=15) as executor:
        # Map each task to the `run_simulation` function
        results = executor.map(run_simulation, *zip(*tasks))  # Unpack the tasks list into separate arguments
        
        # Collect the results in fidelitylist
        fidelitylist = np.array(list(results))
    
    return fidelitylist

# Example usage:

kappalist = np.linspace(0.0, 0.2, 10)
drlist = np.linspace(0.0, 0.2, 10)
fidelitylist=np.array([])
# Run the parallelized simulations
#fidelitylist = parallelize_simulations(kappalist, drlist)
for ka in kappalist:
    for dr in drlist:
        fidelitylist=np.append(fidelitylist,run_simulation(ka, dr))

In [ ]:
display(fidelitylist)

In [ ]:
'''
Calculate fidelities for Dicke states with n=1,2 for N_atoms in [2,6]
'''

Natoms=np.arange(2,7)
Nc=2
nd=1
fidelitiesl=np.array([])
stateslst=np.array([])

for n in Natoms:
    dicke2=generate_dicke_state(n, nd, Nc)
    #display(Dc1.check_ACeffect())
    flist_temp=np.array([])
    for i in range(11):
        Dc1=DickeRaman(Nbits=n, Ncav=Nc, Det=48.0, tdet=0.0, spem=True,cavdecay=True)
        Dc1.construct_ham(pulse1, pulse2);
        Dc1.apply_dissipation(0.0,0.0,0.0);
        Dc1.inject_photon(npx=nd,coherent = False);
        Dc1.apply_ACdispersive(Pa,APulse,Delta0);
        Dc1.apply_controlfield(CPulse);
        Dc1.introduce_globalpiswap(3.0*dur,0.01)
        Dc1.introduce_globalpiswap(6.0*dur,0.01)
        Dc1.introduce_drift(driftA=0.02);
        Dc1.introduce_freqnoise(system="atomic",noise_type="Deterministic",sigma=[0.0,0.0,0.0],noise_freq=20.0);
        sts0=Dc1.simulate_mcwf(tlist,num_cpus=60, ntraj=1,options=options1)
        sts=sts0.states
        flist_temp=np.append(flist_temp,max([fidelity(x,dicke2) for x in sts]))
    #statelst=np.append(stateslst,[sts])
    display(min(flist_temp))
    fidelitiesl=np.append(fidelitiesl,min(flist_temp))
display(fidelitiesl)

Nc=3
nd=2
fidelitiesl2=np.array([])
stateslst2=np.array([])
for n in Natoms:
    dicke2=generate_dicke_state(n, nd, Nc)
    #display(Dc1.check_ACeffect())
    flist_temp=np.array([])
    for i in range(11):
        Dc1=DickeRaman(Nbits=n, Ncav=Nc, Det=48.0, tdet=0.0, spem=True,cavdecay=True)
        Dc1.construct_ham(pulse1, pulse2);
        Dc1.apply_dissipation(0.0,0.0,0.0);
        Dc1.inject_photon(npx=nd,coherent = False);
        Dc1.apply_ACdispersive(Pa,APulse,Delta0);
        Dc1.apply_controlfield(CPulse);
        Dc1.introduce_globalpiswap(3.0*dur,0.01)
        Dc1.introduce_globalpiswap(6.0*dur,0.01)
        Dc1.introduce_drift(driftA=0.02);
        Dc1.introduce_freqnoise(system="atomic",noise_type="Deterministic",sigma=[0.0,0.0,0.0],noise_freq=10.0);
        sts0=Dc1.simulate_mcwf(tlist,num_cpus=60, ntraj=1,options=options1)
        sts=sts0.states
        flist_temp=np.append(flist_temp,max([fidelity(x,dicke2) for x in sts]))
    #statelst=np.append(stateslst,[sts])
    display(min(flist_temp))
    fidelitiesl2=np.append(fidelitiesl2,min(flist_temp))
display(fidelitiesl2)

In [ ]:
from functools import reduce
s1m_list, s2m_list, s1_list, s2_list, s3_list, cav_list=Dc1.return_operators()
s1=s3_list[0]*s1_list[1]
s0=s3_list[1]*s1_list[0]
dicke2=generate_dicke_state(3, 1, 2)
ggg0=tensor([basis(3, 0) for _ in range(3)]+[basis(2,1)])
wst=generate_w_state(3,2)
#display(wst.full())
def density_matrix_to_ket(rho):
    """ Convert a pure density matrix to a ket if possible """
    if (rho * rho).tr() >0.9:  # Purity condition: Tr(rho^2) = 1
        return rho.eigenstates()[1][-1]  # Return the pure eigenvector
    else:
        raise ValueError("The density matrix is not pure. Cannot extract a single ket.")
#display(np.exp(1j*2.93261662)*density_matrix_to_ket(sts[-1]).full())
display(fidelity(sts[-1],dicke2))
#display(fidelity(sts[0],wst*wst.dag()))
#display(np.exp(-1j*2.93261662)*sts[-1].full())
#display(fidelity(np.exp(-1j*2.93261662)*sts[-1],wst*wst.dag()))
#0.9778134057409239
#0.7451573074743248

In [ ]:
import matplotlib.pyplot as plt
display(max([fidelity(x,dicke2) for x in sts]))
f, ax = plt.subplots()
ax.plot(tlist, np.real([expect(dicke2*dicke2.dag(),x) for x in sts]),label=r'fff0',linewidth=3)
ax.plot(tlist, np.real([expect(ggg0*ggg0.dag(),x) for x in sts]),label=r'ggg1',linewidth=3)
ax.legend()
plt.show()

f, ax = plt.subplots()
ax.plot(tlist, Jx,label=r'Jx',linewidth=3)
ax.plot(tlist, Jy,label=r'Jy',linewidth=3)
ax.plot(tlist, Jz,label=r'Jz',linewidth=3)
ax.plot(tlist, Jsq,label=r'Jsq',linewidth=3)
ax.legend()
#ax.plot(tlist, np.real([expect(sum(y for y in s3_list),x) for x in sts]),label=r'0',linewidth=3)
plt.show()

In [ ]:
class e_result:
    def __init__(self,tlist,expect):
        self.times = tlist
        self.expect = expect

Dc1.Bloch_animate(filename='bloch_sphere')